# GridSearch y Pipelines

En este notebook vamos a ver un par de cosas que ya hemos mencionado en algún que otro notebook:
 - **GridSearch**: es una herramienta de optimización que se utiliza para buscar la mejor combinación de hiperparámetros. Consiste en definir una red de valores para cada parámetro, y el objeto se encargará de gestionar la combinación de cada uno de ellos, devolviendo el mejor resultado en cada caso.
 
 - **Pipelines**: son una interfaz para trabajar con combinaciones de objetos de ``sklearn`` como si fueran uno solo. De este modo, podremos diseñar combinaciones de tratamientos de datos con modelos y utilizarlos en nuestros GridSearch, o aplicarles cross validation, como vimos en el notebook pasado. En este caso los implementaremos directamente, en lugar de utilizar una función específica.
 
Para implementar la búsqueda de la combinación de parámetros óptima, vamos a ver 3 estrategias, que irán avanzando de menor a mayor complejidad:

### 1. Método simple

Itera un algoritmo sobre un conjunto de hiperparámetros. Para ello, utilizaremos el objeto ``GridSearchCV``:

In [12]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [13]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV, train_test_split


# Cargamos los datos:
iris = datasets.load_iris()

X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size = 0.2,
                                                   random_state=42)

# Creamos modelo:
svc = svm.SVC()

# Definimos parámetros:
parameters = {
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'C': [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 100],
    'gamma': ['scale', 'auto'],
    'coef0': [-10, -1, 0, 0.1, 0.5, 1, 10, 100]
}

# Nos creamos iterador basado en cross validation:
grid = GridSearchCV(estimator = svc,
                   param_grid = parameters,
                   n_jobs = -1,
                   scoring = 'accuracy',
                   cv = 10)

# Entrenamos sobre train:
grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 100],
                         'coef0': [-10, -1, 0, 0.1, 0.5, 1, 10, 100],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'sigmoid']},
             scoring='accuracy')

In [14]:
grid.best_estimator_.score(X_train, y_train)

0.9666666666666667

In [15]:
# Obtenemos los mejores parámetros:
print("Best estimator:", grid.best_estimator_)
print("Best params:", grid.best_params_)
print("Best score:", grid.best_score_)

Best estimator: SVC(C=0.1, coef0=-10, kernel='linear')
Best params: {'C': 0.1, 'coef0': -10, 'gamma': 'scale', 'kernel': 'linear'}
Best score: 0.9583333333333334


In [16]:
# Y lo probamos sobre test:
best_estimator = grid.best_estimator_
best_estimator.score(X_test, y_test)

1.0

Aquí lo estaríamos haciendo ya bien por fin, ya que dividimos entre train y test. Luego hacemos el cross validation sobre train. Y, finalmente, probamos el desempeño sobre test para verificar que no nos sale nada desorbitado.

### EJERCICIO

Vamos a poner esto en práctica con el último ejemplo que vimos para la selección de variables. Prueba a variar los parámetros partiendo de un algoritmo de regresión logística para predecir si una persona ha sobrevivido o no.

Para que no andes rebuscando en los notebooks, te dejo por aquí el tratamiento común que le habíamos dado el otro día, pero te recomiendo probar por ti mismo a cambiar cosas y crear nuevas variables también:

In [17]:
import pandas as pd
df = pd.read_csv("../../../data/titanic.csv", sep='\t')
df
# El que queda de Embarked, lo vamos a rellenar con "S" porque es el mayoritario.
df['Embarked'] = df['Embarked'].fillna('S')
# En Age, vamos a completar los valores nulos con la media de los valores de Age.
df['Age'] = df['Age'].fillna(df['Age'].mean())
# En Pclass, vamos a convertirlo a string para luego hacer el One Hot Encoding.
df['Pclass'] = df['Pclass'].astype(str)
# Separamos las variables que podemos utilizar para predecir de la que queremos predecir:
y_col = 'Survived'
X_cols = [col for col in df.columns if col not in ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived']]
X = df[X_cols]
y = df[y_col]
# Tratamiento categóricas:
X = pd.get_dummies(X)

In [18]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.3,random_state=42)
# Creamos modelo:
modelo_rl = LogisticRegression(max_iter = 20000, penalty='elasticnet', class_weight='balanced', solver='saga')
# lf = DecisionTreeClassifier()
#svc = svm.SVC()
# Definimos parámetros:
#lr
parameters = {
    'C': [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 100],
    'l1_ratio': [0, 0.25, 0.5, 0.75, 1]
}
#clf
# parameters = {
#    'criterion': ['gini','entropía'],
#    'splitter': ['best','random'],
#    'max_depth': [1, 3, 5, 10],
#    'random_state': [17]
# }
#svc
#parameters = {
#    'kernel': ['linear', 'rbf', 'sigmoid'],
#    'C': [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 100],
#   'gamma': ['scale', 'auto'],
#    'coef0': [-10, -1, 0, 0.1, 0.5, 1, 10, 100]
#}
# Nos creamos iterador basado en cross validation:
grid = GridSearchCV(estimator = modelo_rl,
                   param_grid = parameters,
                   n_jobs = -1,
                   scoring = 'accuracy',
                   cv = 5)
# Entrenamos sobre train:
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced',
                                          max_iter=20000, penalty='elasticnet',
                                          solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 100],
                         'l1_ratio': [0, 0.25, 0.5, 0.75, 1]},
             scoring='accuracy')

In [20]:
grid.best_estimator_.score(X_test, y_test)

0.7446808510638298

In [21]:
grid.best_score_

0.7974025974025974

In [22]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [23]:
log_reg.score(X_test, y_test)

0.7659574468085106

### 2. Forma pro


En este caso, además, introduciremos los pipelines, creándolos utilizando directamente su constructor ``Pipeline(steps=[])``. Si te fijas bien en el código, con este método podemos comparar diferentes algoritmos, con sus diferentes combinaciones de parámetros, gracias a estos pipelines. La sintáxis es el objeto ``Pipeline``, que recibe como parámetros ``steps``, que es una lista de tuplas, donde la primera se corresponde con el nombre que le das al paso, y el segundo el obejto que se ejecuta en ese paso:

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

pipe = Pipeline(steps=[('classifier', DecisionTreeClassifier())])

logistic_params = {
    'classifier': [LogisticRegression()],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__C': np.arange(0, 4, 0.5)
}

decision_tree_params = {
    'classifier': [DecisionTreeClassifier()],
    'classifier__max_depth': [10, 8, 5, 2],
    'classifier__criterion': ['entropy', 'gini']
}

svc_params = {
    'classifier': [svm.SVC()],
    'classifier__kernel': ['linear', 'rbf', 'sigmoid']
}

search_space = [logistic_params, decision_tree_params, svc_params]
# search_space = [logistic_params, decision_tree_params]

grid = GridSearchCV(pipe,
                   search_space,
                   cv=10,
                   n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('classifier',
                                        DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid=[{'classifier': [LogisticRegression()],
                          'classifier__C': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5]),
                          'classifier__penalty': ['l1', 'l2']},
                         {'classifier': [DecisionTreeClassifier()],
                          'classifier__criterion': ['entropy', 'gini'],
                          'classifier__max_depth': [10, 8, 5, 2]},
                         {'classifier': [SVC(kernel='linear')],
                          'classifier__kernel': ['linear', 'rbf', 'sigmoid']}])

In [25]:
print(grid.best_estimator_)
print(grid.best_params_)
print(grid.best_score_)

Pipeline(steps=[('classifier', SVC(kernel='linear'))])
{'classifier': SVC(kernel='linear'), 'classifier__kernel': 'linear'}
0.8336363636363636


In [26]:
grid.score(X_test, y_test)

0.7446808510638298

In [27]:
pd.DataFrame({'predict': grid.predict(X_test), 'real': y_test}).join(X)

,predict,real,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
96,0,0,71.000000,0,0,34.6542,1,0,0,0,1,1,0,0
69,0,0,26.000000,2,0,8.6625,0,0,1,0,1,0,0,1
82,1,1,28.141508,0,0,7.7875,0,0,1,1,0,0,1,0
76,0,0,28.141508,0,0,7.8958,0,0,1,0,1,0,0,1
114,1,0,17.000000,0,0,14.4583,0,0,1,1,0,1,0,0
29,0,0,28.141508,0,0,7.8958,0,0,1,0,1,0,0,1
94,0,0,59.000000,0,0,7.2500,0,0,1,0,1,0,0,1
132,1,0,47.000000,1,0,14.5000,0,0,1,1,0,0,0,1
93,0,0,26.000000,1,2,20.5750,0,0,1,0,1,0,0,1
139,0,0,24.000000,0,0,79.2000,1,0,0,0,1,1,0,0


### EJERCICIO

Bueno pues, para que podamos comparar qué nos ofrece una versión frente a otra, vamos a volver sobre el mismo ejemplo del Titanic para probar estas técnicas.

1. Utiliza el GridSearch para encontrar la combinación de parámetros que mejores resultados ofrece, probando, esta vez, con otros algoritmos como Árbol de decisión y SVC.

Utiliza la misma separación train/test de antes. También sería conveniente que utilizaras una semilla apra tener cierta reproducibilidad de los datos, es decir, que vuelva a salir lo mismo si vuelves a ejecutar:

In [28]:
import pandas as pd
df = pd.read_csv("../../../data/titanic.csv", sep='\t')
df

# El que queda de Embarked, lo vamos a rellenar con "S" porque es el mayoritario.
df['Embarked'] = df['Embarked'].fillna('S')

# En Age, vamos a completar los valores nulos con la media de los valores de Age.
df['Age'] = df['Age'].fillna(df['Age'].mean())

# En Pclass, vamos a convertirlo a string para luego hacer el One Hot Encoding.
df['Pclass'] = df['Pclass'].astype(str)

# Separamos las variables que podemos utilizar para predecir de la que queremos predecir:
y_col = 'Survived'

X_cols = [col for col in df.columns if col not in ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived']]

X = df[X_cols]
y = df[y_col]

# Tratamiento categóricas:
X = pd.get_dummies(X)

### 3. Next Level

Finalmente, tenemos otra posible implementación, donde guardamos cada uno de los grids para tener un mayor control de cada modelo:

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [56]:
# Comenzamos definiendo los pipelines principales:
reg_log = Pipeline(steps = [
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler()),
    ("reglog", LogisticRegression())
])

svc = Pipeline([
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler()),
    ("selectkbest", SelectKBest()),
    ("svc", svm.SVC())
])

decision_tree = DecisionTreeClassifier()

# Y definimos sus parámetros:
re_log_param = {
    "imputer__strategy": ['mean', 'median', 'most_frequent'],
    "reglog__penalty": ["l1", "l2"],
    "reglog__C": np.arange(0, 4, 0.5)
}

svc_param = {
    "imputer__strategy": ['mean', 'median', 'most_frequent'],
    "selectkbest__k": [1, 2, 3],
    "svc__C": np.arange(0.1, 0.9, 0.1),
    "svc__kernel": ['linear', 'poly', 'rbf']
}

decision_tree_params = {
    'max_depth': [10, 100, 500, 1000],
    'criterion': ['gini', 'entropy']
}

# Nos creamos los grids de cada uno:
gs_reg_log = GridSearchCV(reg_log,
                         re_log_param,
                         cv = 10,
                         scoring='accuracy',
                         n_jobs=-1,
                         verbose=1)

gs_svm = GridSearchCV(svc,
                         svc_param,
                         cv = 10,
                         scoring='accuracy',
                         n_jobs=-1,
                         verbose=1)

gs_decision_tree = GridSearchCV(decision_tree,
                         decision_tree_params,
                         cv = 10,
                         scoring='accuracy',
                         n_jobs=-1,
                         verbose=1)

grids = {
    "gs_reg_log": gs_reg_log,
    "gs_svm": gs_svm,
    "gs_rand_forest": gs_decision_tree
}
#gs_reg_log.fit(X_train, y_train)
#print("Best estimator:", gs_reg_log.best_estimator_)
#print("Best params:", gs_reg_log.best_params_)
#print("Best score:", gs_reg_log.best_score_)

In [31]:
d = {'a':1, 'b':2, 'c': 'hola', 'f':17}
d['f']

17

In [32]:
for a, b in d.items():
    print(f"Clave: {a}, Valor: {b}")

Clave: a, Valor: 1
Clave: b, Valor: 2
Clave: c, Valor: hola
Clave: f, Valor: 17


In [57]:
for nombre, grid_search in grids.items():
    print(f"Calculando {nombre}")
    grid_search.fit(X_train, y_train)

Calculando gs_reg_log
Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:    4.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Calculando gs_svm
Fitting 10 folds for each of 216 candidates, totalling 2160 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 1200 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 2160 out of 2160 | elapsed:    8.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Calculando gs_rand_forest
Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    0.1s finished


In [24]:
grids.items()

dict_items([('gs_reg_log', GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('reglog', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'imputer__strategy': ['mean', 'median',
                                               'most_frequent'],
                         'reglog__C': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5]),
                         'reglog__penalty': ['l1', 'l2']},
             scoring='accuracy', verbose=1)), ('gs_svm', GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selectkbest', SelectKBest()),
                                       ('svc', SVC())]),
             n_jobs=-1,
             param_grid={'selectkbest__k': [1, 2, 3],
                         'svc__C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]),
    

In [46]:
grids['gs_reg_log'].best_estimator_.score(X_test, y_test)

0.7659574468085106

In [58]:
l = []
for i, j in grids.items():
    l.append((i, j.best_score_))
l

[('gs_reg_log', 0.8163636363636364),
 ('gs_svm', 0.8336363636363636),
 ('gs_rand_forest', 0.77)]

In [59]:
best_grids = [(i, j.best_score_, j.best_params_) for i, j in grids.items()]

best_grids = pd.DataFrame(best_grids, columns = ['Grid', 'Best score', 'Best params'])
best_grids.sort_values(by='Best score', ascending=False)

,Grid,Best score
1,gs_svm,0.833636
0,gs_reg_log,0.816364
2,gs_rand_forest,0.770000


In [60]:
print("Best estimator:", gs_svm.best_estimator_)
print("Best params:", gs_svm.best_params_)
print("Best score:", gs_svm.best_score_)

Best estimator: Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('selectkbest', SelectKBest(k=1)),
                ('svc', SVC(C=0.1, kernel='linear'))])
Best params: {'imputer__strategy': 'mean', 'selectkbest__k': 1, 'svc__C': 0.1, 'svc__kernel': 'linear'}
Best score: 0.8336363636363636


In [60]:
print("Best estimator:", gs_svm.best_estimator_)
print("Best params:", gs_svm.best_params_)
print("Best score:", gs_svm.best_score_)

Best estimator: Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('selectkbest', SelectKBest(k=1)),
                ('svc', SVC(C=0.1, kernel='linear'))])
Best params: {'imputer__strategy': 'mean', 'selectkbest__k': 1, 'svc__C': 0.1, 'svc__kernel': 'linear'}
Best score: 0.8336363636363636


In [163]:
gs_svm.best_estimator_

Pipeline(steps=[('scaler', MinMaxScaler()), ('selectkbest', SelectKBest(k=7)),
                ('svc', SVC(C=0.4))])

In [164]:
gs_reg_log.best_estimator_

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler()),
                ('reglog', LogisticRegression(C=2.0, solver='liblinear'))])

In [49]:
# Obtenemos mejores resultados:
estimador = grids['gs_svm'].best_estimator_
estimador = gs_svm.best_estimator_

estimador.score(X_test, y_test)

0.7446808510638298

In [50]:
# Podemos predecir:
estimador.predict(X_test)

array([0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0], dtype=int64)

In [32]:
# Extraemos los nombres de las variables:
iris['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [51]:
estimador['selectkbest'].scores_

array([8.91578151e-01, 3.53252705e-01, 1.17238473e-01, 1.01363643e-02,
       4.04137645e-01, 1.75294889e-02, 4.11273781e-01, 6.94578704e+01,
       6.94578704e+01, 1.82535869e-01, 9.90160569e-01, 1.00259446e-01])

In [52]:
# Sacamos los pvalores de cada uno
k_seleccionadas = estimador['selectkbest'].k
pd.DataFrame({'score': estimador['selectkbest'].scores_, 'variable': X.columns}).sort_values(by="score", ascending=False).iloc[:k_seleccionadas]['variable'].values

array(['Sex_female'], dtype=object)

Si queremos guardar el modelo, podemos ayudarnos de **pickle**:

In [53]:
import pickle

# Escribir
with open('carpeta/finished_model.model', 'wb') as archivo_salida:
    pickle.dump(estimador, archivo_salida)

In [6]:
import pickle

# Leer
with open('carpeta/finished_model.model', 'rb') as archivo_entrada:
    pipeline_importado = pickle.load(archivo_entrada)

In [34]:
import pickle

# Escribir
with open('carpeta/diccionario_modelos', 'wb') as archivo_salida:
    pickle.dump(grids, archivo_salida)

In [35]:
import pickle

# Leer
with open('carpeta/diccionario_modelos', 'rb') as archivo_entrada:
    grids_leido = pickle.load(archivo_entrada)

Una vez importado, ya podemos utilizarlo como si lo acabásemos de crear:

In [43]:
grids_leido['gs_reg_log'].best_params_

{'imputer__strategy': 'mean', 'reglog__C': 1.0, 'reglog__penalty': 'l2'}

In [7]:
pipeline_importado

Pipeline(steps=[('scaler', StandardScaler()), ('selectkbest', SelectKBest(k=1)),
                ('svc', SVC(C=0.1, kernel='linear'))])

In [8]:
pipeline_importado.n_features_in_

12

In [9]:
new_flowers = np.array([[6.9, 3.1, 5.1, 2.3, 6.9, 3.1, 5.1, 2.3, 6.9, 3.1, 5.1, 2.3],
                        [5.8, 2.7, 3.9, 1.2, 6.9, 3.1, 5.1, 2.3, 4.3, 5.1, 5.1, 3.2]])

In [11]:
pipeline_importado['selectkbest'].scores_

array([8.91578151e-01, 3.53252705e-01, 1.17238473e-01, 1.01363643e-02,
       4.04137645e-01, 1.75294889e-02, 4.11273781e-01, 6.94578704e+01,
       6.94578704e+01, 1.82535869e-01, 9.90160569e-01, 1.00259446e-01])

In [44]:
pipeline_importado.predict(new_flowers)

array([0, 0], dtype=int64)

### EJERCICIO

Para terminar este ejemplo, y que se entienda de la mejor manera posible, vamos a repetir el experimento con el mismo dataset y el mismo objetivo. La única diferecia será que, esta vez, nos centraremos en añadir pasos a los pipelines principales (como selectores de variables o escalados).

También nos interesará saber cuál es la mejor combinación para cada modelo. Utiliza, al menos, 3 de estos modelos: LogisticRegression, DecisionTreeClasiffier, LinearSVC, SVC y KNN; y realiza la combinación variando, al menos, 2 parámetros diferentes para cada uno.

Además, finaliza guardando el modelo que mejor resultado te ofrezca para utilizarlo en cualquier otro momento.

Como hemos comentado antes, utiliza la misma separación train/test de antes. También sería conveniente que utilizaras una semilla.

In [155]:
import pandas as pd
df = pd.read_csv("../../../data/titanic.csv", sep='\t')
df

# El que queda de Embarked, lo vamos a rellenar con "S" porque es el mayoritario.
df['Embarked'] = df['Embarked'].fillna('S')

# En Age, vamos a completar los valores nulos con la media de los valores de Age.
df['Age'] = df['Age'].fillna(df['Age'].mean())

# En Pclass, vamos a convertirlo a string para luego hacer el One Hot Encoding.
df['Pclass'] = df['Pclass'].astype(str)

# Separamos las variables que podemos utilizar para predecir de la que queremos predecir:
y_col = 'Survived'
X_cols = [col for col in df.columns if col not in ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived']]

X = df[X_cols]
y = df[y_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=42)

# Tratamiento categóricas:
X = pd.get_dummies(X)

In [156]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [157]:
# Comenzamos definiendo los pipelines principales:
reg_log = Pipeline([
    ("imputer", SimpleImputer()),
    ("scaler", MinMaxScaler()),
    ("reglog", LogisticRegression())
])
svc = Pipeline([
    ("scaler", MinMaxScaler()),
    ("selectkbest", SelectKBest()),
    ("svc", svm.SVC())
])
decision_tree = DecisionTreeClassifier()
knear = Pipeline([
    ("scaler", MinMaxScaler()),
    ("selectkbest", SelectKBest()),
    ("knn", KNeighborsClassifier())
])
# Y definimos sus parámetros:
re_log_param = {
    "imputer__strategy": ['mean', 'median', 'most_frequent'],
    "reglog__solver": ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    "reglog__penalty": ["l1", "l2", "elasticnet"],
    "reglog__C": np.arange(0, 4, 0.5)
}
svc_param = {
    "selectkbest__k": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "svc__C": np.arange(0.1, 0.9, 0.1),
    "svc__kernel": ['linear', 'poly', 'rbf']
}
decision_tree_params = {
    "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "criterion": ['gini', 'entropy']
}
knear_params = {
    "knn__n_neighbors": np.arange(1, 33, 2),
    "knn__weights": ['uniform', 'distance'],
    "knn__algorithm": ['ball_tree', 'kd_tree', 'brute']
}

# Nos creamos los grids de cada uno:
gs_reg_log = GridSearchCV(reg_log,
                         re_log_param,
                         cv = 10,
                         scoring='accuracy',
                         n_jobs=-1,
                         verbose=1)
gs_svm = GridSearchCV(svc,
                         svc_param,
                         cv = 10,
                         scoring='accuracy',
                         n_jobs=-1,
                         verbose=1)
gs_decision_tree = GridSearchCV(decision_tree,
                         decision_tree_params,
                         cv = 10,
                         scoring='accuracy',
                         n_jobs=-1,
                         verbose=1)
gs_knn = GridSearchCV(knear,
                      knear_params,
                      cv = 10,
                      scoring = 'accuracy',
                      n_jobs = -1,
                      verbose = 1)
grids = {
    "gs_reg_log": gs_reg_log,
    "gs_svm": gs_svm,
    "gs_rand_forest": gs_decision_tree,
    "gs_knn": gs_knn
}

In [158]:
for nombre, grid_search in grids.items():
    print(f"Calculando {nombre}")
    grid_search.fit(X_train, y_train)

Calculando gs_reg_log
Fitting 10 folds for each of 360 candidates, totalling 3600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 2160 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 3600 out of 3600 | elapsed:   11.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Calculando gs_svm
Fitting 10 folds for each of 240 candidates, totalling 2400 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 1200 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 2400 out of 2400 | elapsed:    8.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Calculando gs_rand_forest
Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Calculando gs_knn
Fitting 10 folds for each of 96 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:    3.1s finished


In [159]:
l = []
for i, j in grids.items():
    l.append((i, j.best_score_))
l

[('gs_reg_log', 0.791025641025641),
 ('gs_svm', 0.8006410256410256),
 ('gs_rand_forest', 0.7833333333333334),
 ('gs_knn', 0.8076923076923077)]

In [161]:
print("Best estimator:", gs_knn.best_estimator_)
print("Best params:", gs_knn.best_params_)
print("Best score:", gs_knn.best_score_)

Best estimator: Pipeline(steps=[('scaler', MinMaxScaler()), ('selectkbest', SelectKBest()),
                ('knn',
                 KNeighborsClassifier(algorithm='ball_tree', n_neighbors=9))])
Best params: {'knn__algorithm': 'ball_tree', 'knn__n_neighbors': 9, 'knn__weights': 'uniform'}
Best score: 0.8076923076923077


In [162]:
gs_knn.best_estimator_.score(X_test, y_test)

0.84375

Otros ejemplos de pipelines:

In [88]:
from sklearn.feature_selection import SelectFromModel

In [105]:
clf = DecisionTreeClassifier()
lr = LogisticRegression()

reg_log = Pipeline(steps = [
    ("ohe", OneHotEncoder(handle_unknown='ignore', sparse=False)),
    ("imputer", SimpleImputer()),
    ("selectmodel", SelectFromModel(estimator=clf)),
    ("scaler", StandardScaler()),
    ("reglog", LogisticRegression())
])

# Y definimos sus parámetros:
re_log_param = {
    "selectmodel__estimator": [clf, lr], 
    "imputer__strategy": ['mean', 'median', 'most_frequent'],
    "reglog__penalty": ["l1", "l2"],
    "reglog__C": np.arange(0, 4, 0.5)
}

In [106]:
# Nos creamos los grids de cada uno:
gs_reg_log = GridSearchCV(reg_log,
                         re_log_param,
                         cv = 10,
                         scoring='accuracy',
                         n_jobs=-1,
                         verbose=1)

In [107]:
gs_reg_log.fit(X_train, y_train)

Fitting 10 folds for each of 96 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:    5.8s finished


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('ohe',
                                        OneHotEncoder(handle_unknown='ignore',
                                                      sparse=False)),
                                       ('imputer', SimpleImputer()),
                                       ('selectmodel',
                                        SelectFromModel(estimator=DecisionTreeClassifier())),
                                       ('scaler', StandardScaler()),
                                       ('reglog', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'imputer__strategy': ['mean', 'median',
                                               'most_frequent'],
                         'reglog__C': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5]),
                         'reglog__penalty': ['l1', 'l2'],
                         'selectmodel__estimator': [DecisionTreeClassifier(),
                                             